# DSAI-LSTM Subtractor

## Import package

In [62]:
from keras.models import Sequential
from keras import layers
from keras.layers import LSTM, TimeDistributed, Dense, RepeatVector, Activation, BatchNormalization,Bidirectional
from keras.models import load_model
import numpy as np
from six.moves import range

## Parameters Config

In [48]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

In [49]:
TRAINING_SIZE = 80000
DIGITS = 3
REVERSE = False
MAXLEN = DIGITS + 1 + DIGITS+6
chars = '0123456789-+xyzabc '
RNN = layers.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 128
LAYERS = 1

## Data Generation

- 隨機生成兩數相減或相加字串 <br>
  減法:若兩數第1,2,3個數字相減小於0，則分別在字串後串接x,y,z <br>
  加法:若兩數第1,2,3個數字相加大於10，則分別在字串後串接a,b,c


In [72]:
%%time
data = []
label = []
seen = set()

print('Generating data...')
while len(data) < TRAINING_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789')) for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    if(a<b):
        a,b = b,a
    
    operator = np.random.choice(list('+-'))
    
#     q = '{}-{}'.format(a, b)
    q = str(a) + operator + str(b)    
    
    if(operator == "-"):
        for i in range(len(str(b))):
            if(int(str(a)[-i-1]) - int(str(b)[-i-1]) < 0 ):
                if(i==0):
                    q+= "z"
                elif(i == 1):
                    q+= "y"
                else:
                    q+= "x"
                    
    elif(operator == '+'):
        for i in range(len(str(b))):
            if(int(str(a)[-i-1]) + int(str(b)[-i-1]) > 10 ):
                if(i==0):
                    q+= "c"
                elif(i == 1):
                    q+= "b"
                else:
                    q+= "a"


    if(q not in seen):
        query = q + ' ' * (MAXLEN - len(q))
        seen.add(query)
        data.append(query)
        
        if(operator == "+"):
            ans = str(a+b)
        else:
            ans = str(a-b)
        
        ans += ' '* (DIGITS + 1 - len(ans))
        label.append(ans)
        
    
# print(data)
# print(label)

Generating data...
Wall time: 7.5 s


## Processing

In [73]:
class CharacterTable(object):
    def __init__(self, chars):
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))
    
    def encode(self, C, num_rows):
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x
    
    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return "".join(self.indices_char[i] for i in x)

In [74]:
ctable = CharacterTable(chars)

print('Vectorization...')
x = np.zeros((len(data), MAXLEN, len(chars)))
y = np.zeros((len(label), DIGITS + 1, len(chars)))
for i, sentence in enumerate(data):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(label):
    y[i] = ctable.encode(sentence, DIGITS + 1)
    
# print(x.shape)
# print(y.shape)
# print(y)

Vectorization...


In [75]:
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# train_test_split
train_x = x[:40000]
train_y = y[:40000]
test_x = x[40000:]
test_y = y[40000:]

split_at = len(train_x) - len(train_x) // 10
print(len(train_x))

(x_train, x_val) = train_x[:split_at], train_x[split_at:]
(y_train, y_val) = train_y[:split_at], train_y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

print('Testing Data:')
print(test_x.shape)
print(test_y.shape)

40000
Training Data:
(36000, 13, 19)
(36000, 4, 19)
Validation Data:
(4000, 13, 19)
(4000, 4, 19)
Testing Data:
(40000, 13, 19)
(40000, 4, 19)


## Build Model

In [56]:
print('Build model...')
model = Sequential()

model.add(BatchNormalization(input_shape =(MAXLEN,len(chars))))
model.add(Bidirectional(RNN(HIDDEN_SIZE),merge_mode='concat'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(layers.RepeatVector(DIGITS + 1))
model.add(RNN(HIDDEN_SIZE, return_sequences=True))
model.add(layers.TimeDistributed(layers.Dense(len(chars))))
model.add(layers.Activation('softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_11 (Batc (None, 13, 19)            76        
_________________________________________________________________
bidirectional_6 (Bidirection (None, 256)               151552    
_________________________________________________________________
batch_normalization_12 (Batc (None, 256)               1024      
_________________________________________________________________
activation_11 (Activation)   (None, 256)               0         
_________________________________________________________________
repeat_vector_6 (RepeatVecto (None, 4, 256)            0         
_________________________________________________________________
lstm_12 (LSTM)               (None, 4, 128)            197120    
_________________________________________________________________
time_distributed_6 (TimeDist (None, 4, 19)             2451  

## Training

In [57]:
for iteration in range(40):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val))
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)



--------------------------------------------------
Iteration 0
Train on 36000 samples, validate on 4000 samples
Epoch 1/1
36000/36000 [==============================] - 22s 617us/step - loss: 1.2180 - acc: 0.5899 - val_loss: 0.9029 - val_acc: 0.6785
Q 501+84        T 585  ☒ 588 
Q 248-9z        T 239  ☒ 279 
Q 871-3z        T 868  ☒ 888 
Q 775-5         T 770  ☒ 777 
Q 140+67        T 207  ☒ 168 
Q 88+1          T 89   ☑ 89  
Q 95+63b        T 158  ☒ 157 
Q 805-8z        T 797  ☒ 895 
Q 882+291b      T 1173 ☒ 117 
Q 905+75        T 980  ☒ 999 

--------------------------------------------------
Iteration 1
Train on 36000 samples, validate on 4000 samples
Epoch 1/1
36000/36000 [==============================] - 18s 508us/step - loss: 0.5765 - acc: 0.8112 - val_loss: 0.4010 - val_acc: 0.8714
Q 108-86y       T 22   ☒ 42  
Q 244+58c       T 302  ☒ 392 
Q 616-1         T 615  ☑ 615 
Q 490+42b       T 532  ☒ 522 
Q 27-3          T 24   ☑ 24  
Q 493-5z        T 488  ☑ 488 
Q 92+6          T 

36000/36000 [==============================] - 17s 467us/step - loss: 0.0275 - acc: 0.9922 - val_loss: 0.0484 - val_acc: 0.9835
Q 898+407ca     T 1305 ☑ 1305
Q 562-68z       T 494  ☑ 494 
Q 63-3          T 60   ☑ 60  
Q 952+47        T 999  ☒ 909 
Q 40+2          T 42   ☑ 42  
Q 972+9c        T 981  ☑ 981 
Q 118-3         T 115  ☑ 115 
Q 98-6          T 92   ☑ 92  
Q 871-3z        T 868  ☑ 868 
Q 3+1           T 4    ☑ 4   

--------------------------------------------------
Iteration 14
Train on 36000 samples, validate on 4000 samples
Epoch 1/1
36000/36000 [==============================] - 17s 466us/step - loss: 0.0218 - acc: 0.9937 - val_loss: 0.0410 - val_acc: 0.9862
Q 37+9c         T 46   ☑ 46  
Q 53+11         T 64   ☑ 64  
Q 63-24z        T 39   ☑ 39  
Q 36+7c         T 43   ☑ 43  
Q 402-89zy      T 313  ☑ 313 
Q 670+9         T 679  ☑ 679 
Q 367-4         T 363  ☑ 363 
Q 654-1         T 653  ☑ 653 
Q 205-24y       T 181  ☑ 181 
Q 291+276b      T 567  ☒ 467 

-------------------

36000/36000 [==============================] - 17s 482us/step - loss: 0.0055 - acc: 0.9987 - val_loss: 0.0272 - val_acc: 0.9917
Q 962-3z        T 959  ☑ 959 
Q 80+76b        T 156  ☑ 156 
Q 68+18c        T 86   ☑ 86  
Q 26-20         T 6    ☑ 6   
Q 8-5           T 3    ☑ 3   
Q 437+350       T 787  ☒ 767 
Q 582-3z        T 579  ☑ 579 
Q 747-265y      T 482  ☑ 482 
Q 550-28z       T 522  ☑ 522 
Q 31-23z        T 8    ☑ 8   

--------------------------------------------------
Iteration 27
Train on 36000 samples, validate on 4000 samples
Epoch 1/1
36000/36000 [==============================] - 17s 486us/step - loss: 0.0042 - acc: 0.9990 - val_loss: 0.0226 - val_acc: 0.9930
Q 571+18        T 589  ☑ 589 
Q 896+787cba    T 1683 ☒ 1763
Q 6+1           T 7    ☑ 7   
Q 741-3z        T 738  ☑ 738 
Q 119+79c       T 198  ☑ 198 
Q 7+2           T 9    ☑ 9   
Q 777-0         T 777  ☑ 777 
Q 743-557zy     T 186  ☑ 186 
Q 275-20        T 255  ☑ 255 
Q 248-9z        T 239  ☑ 239 

-------------------

36000/36000 [==============================] - 18s 495us/step - loss: 0.0017 - acc: 0.9996 - val_loss: 0.0176 - val_acc: 0.9944
Q 8-4           T 4    ☑ 4   
Q 935-8z        T 927  ☑ 927 
Q 700-332zy     T 368  ☑ 368 
Q 493-5z        T 488  ☑ 488 
Q 838+114c      T 952  ☑ 952 
Q 10+5          T 15   ☑ 15  
Q 8-4           T 4    ☑ 4   
Q 237+2         T 239  ☑ 239 
Q 333-224z      T 109  ☑ 109 
Q 375-7z        T 368  ☑ 368 


## Save Model

In [58]:
model.save('BiLSTM.h5') 

## Load Model

In [60]:
model = load_model('BiLSTM.h5')

## Validation

In [76]:
right = 0
preds = model.predict_classes(test_x, verbose=0)
for i in range(len(preds)):
    q = ctable.decode(test_x[i])
    correct = ctable.decode(test_y[i])
    guess = ctable.decode(preds[i], calc_argmax=False)
#     print('Q', q[::-1] if REVERSE else q, end=' ')
#     print('T', correct, end=' ')
    if correct == guess:
#         print(colors.ok + '☑' + colors.close, end=' ')
        right += 1
#     else:
#         print(colors.fail + '☒' + colors.close, end=' ')
#     print(guess)
print("MSG : Accuracy is {}".format(right / len(preds)))


MSG : Accuracy is 0.981425
